In [1]:
import numpy as np
import pandas as pd
import cvxpy as cp
from sklearn import svm

### Purpose
I'd like to better understand the support vector machine and the formulation and optimization of the dual problem central to it. In the following I make my own SVM classifier with a toy dataset in order to do some of this exploration.

### Load up and normalize the data
Here there are data for individuals' income, credit limit, credit rating, etc. and a few binary variables.  We can try to use a few of these features to predict one of the binary variables.  

In [2]:
credit_df = pd.read_table('Credit.dat',
                         delimiter=' ',
                         usecols=np.arange(1,13))
credit_df.head()

,Income,Limit,Rating,Cards,Age,Education,Sex,Student,Married,Balance,Caucasian,Asian
0,14.891,3606.0,283.0,2.0,34.0,11.0,0.0,0.0,1.0,333.0,1.0,0.0
1,106.025,6645.0,483.0,3.0,82.0,15.0,1.0,1.0,1.0,903.0,0.0,1.0
2,104.593,7075.0,514.0,4.0,71.0,11.0,0.0,0.0,0.0,580.0,0.0,1.0
3,148.924,9504.0,681.0,3.0,36.0,11.0,1.0,0.0,0.0,964.0,0.0,1.0
4,55.882,4897.0,357.0,2.0,68.0,16.0,0.0,0.0,1.0,331.0,1.0,0.0


I've normalized the columns such that each value is replaced by its difference from the mean, scaled by the spread of the data in that column

In [3]:
#  standardize all columns
credit_df_norm = (credit_df - credit_df.mean())/(credit_df.std())

In [4]:
credit_df_norm.head()

,Income,Limit,Rating,Cards,Age,Education,Sex,Student,Married,Balance,Caucasian,Asian
0,-0.860505,-0.489386,-0.464957,-0.698255,-1.256101,-0.783948,-1.034339,-0.332916,0.794400,-0.406768,1.003756,-0.584317
1,1.725276,0.827225,0.827667,0.030993,1.526539,0.495967,0.964384,2.996248,0.794400,0.833013,-0.993768,1.707122
2,1.684646,1.013518,1.028023,0.760241,0.888851,-0.783948,-1.034339,-0.332916,-1.255665,0.130471,-0.993768,1.707122
3,2.942467,2.065853,2.107363,0.030993,-1.140158,-0.783948,0.964384,-0.332916,-1.255665,0.965691,-0.993768,1.707122
4,0.302549,0.069925,0.013314,-0.698255,0.714936,0.815946,-1.034339,-0.332916,0.794400,-0.411118,1.003756,-0.584317


some columns are already either 0 or 1, and I'm going to leave these as is

In [5]:
#  revert columns that range from 0 to 1
for col in credit_df.iteritems():
    if ((col[1].max() - col[1].min()) == 1.0):
        credit_df_norm[col[0]] = credit_df[col[0]]

In [6]:
credit_df_norm.head()

,Income,Limit,Rating,Cards,Age,Education,Sex,Student,Married,Balance,Caucasian,Asian
0,-0.860505,-0.489386,-0.464957,-0.698255,-1.256101,-0.783948,0.0,0.0,1.0,-0.406768,1.0,0.0
1,1.725276,0.827225,0.827667,0.030993,1.526539,0.495967,1.0,1.0,1.0,0.833013,0.0,1.0
2,1.684646,1.013518,1.028023,0.760241,0.888851,-0.783948,0.0,0.0,0.0,0.130471,0.0,1.0
3,2.942467,2.065853,2.107363,0.030993,-1.140158,-0.783948,1.0,0.0,0.0,0.965691,0.0,1.0
4,0.302549,0.069925,0.013314,-0.698255,0.714936,0.815946,0.0,0.0,1.0,-0.411118,1.0,0.0


### make training and target arrays

since we have 400 data points to work with, I'll use half to train and half to validate

In [7]:
target='Married'
training_points = 200
train_df = credit_df_norm.drop(target, axis=1).loc[:training_points]
train_target = credit_df_norm[target].replace(0, -1).loc[:training_points]

### definition of the problem

given data points $\vec x_j \in \mathbb{R}^{1 \times N}$ and targets $y_j = \pm 1$, where $j = 1, \dots, M$, find the maximum-margin hyperplane that separates the two classes ($y_j = 1$ and $y_j = -1$).  

Let $\vec w$ be the vector normal to the hyperplane.  We want to find $\vec w$ that satisfies 

$$ y_j (\vec w \cdot \vec x_j + b) \geq 1 $$

The dual formulation of the above is equivalent to maximizing the following over the multipliers $\vec \alpha$:

$$ L(\vec \alpha) = \vec y \cdot \vec \alpha  - \frac 1 2  \vec \alpha K \vec \alpha^T$$

subject to the constraints $\sum_{j=1}^M \alpha_j = 0$ and $y_j \alpha_j \geq 0$.  The matrix $K$ defines the kernel of the SVM; I've chosen $K_{jk} = k(\vec x_j, \vec x_k) = \vec x_j \cdot \vec x_k$.  The parameters of the plane are recovered from $\vec w = \vec \alpha \cdot \vec x$ and $b = y_j = \vec w \cdot \alpha_j$ for $j$ such that $\alpha_j \neq 0$

for reference see eq(1) from:
https://arxiv.org/pdf/1307.0471.pdf

### Make the kernel matrix
since I've defined the kernel as a symmetric bilinear form, the kernel matrix defining the transformation should also be symmetric.  Further, since $ k (\vec x_j, \vec x_k) \equiv \vec x_j \cdot \vec x_k$ the transformation (and thus the matrix) is positive semidefinite ($x_j^i$ admits entries equal to zero).  This is a well known result for linear kernels.  

We can construct the $K$ matrix as follows.  

In [8]:
k_value = np.array(train_df @ train_df.T + np.identity(len(train_target))*1e-12)

the trick here is to add a small constant value along $K$'s diagonal to ensure that small numerical errors that get propagated while calculating eigenvalues don't trigger warnings that our $K$ isn't actually PSD.

In [9]:
k_value

array([[ 5.04153742, -4.94105406, -3.50945918, ...,  1.20452353,
         3.5809826 ,  0.51992661],
       [-4.94105406, 10.61708724,  6.69604535, ..., -2.7768132 ,
        -3.20264287, -0.79965593],
       [-3.50945918,  6.69604535,  7.92170028, ..., -3.7405506 ,
        -0.21268301, -2.95536785],
       ...,
       [ 1.20452353, -2.7768132 , -3.7405506 , ...,  6.70215302,
        -0.44118489, -0.08110758],
       [ 3.5809826 , -3.20264287, -0.21268301, ..., -0.44118489,
         6.93467274, -1.80715243],
       [ 0.51992661, -0.79965593, -2.95536785, ..., -0.08110758,
        -1.80715243,  4.35018343]])

we can check to see that $K$ is indeed positive semidefinite by computing a cholesky decomposition—`cholesky()` from numpy's linalg can do that for us.  If it exists, $K$ must be at least PSD

In [10]:
#  check PSD
np.linalg.cholesky(k_value)

array([[ 2.24533682e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [-2.20058480e+00,  2.40302180e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [-1.56299899e+00,  1.35518268e+00,  1.90845863e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       ...,
       [ 5.36455609e-01, -6.64287416e-01, -1.04893038e+00, ...,
         1.02275371e-06,  0.00000000e+00,  0.00000000e+00],
       [ 1.59485320e+00,  1.27742011e-01,  1.10400988e+00, ...,
        -2.46664055e-08,  1.03358295e-06,  0.00000000e+00],
       [ 2.31558403e-01, -1.20719684e-01, -1.27319767e+00, ...,
         2.83861383e-10, -5.83309832e-09,  1.01751738e-06]])

### set up and minimize the dual function given the constraints

`cvxpy` is a package that allows the user to express a convex optimization problem in a readable form, converts it into a form that can be used to call a solver, solves, and then translates the result again into a readable form.  I can use `cvxpy`'s tools to encode the dual problem that I present above.

In [11]:
alpha = cp.Variable(shape=train_target.shape)

beta = cp.multiply(alpha, train_target) # to simplify notation

K = cp.Parameter(shape=k_value.shape, PSD=True, value=k_value)

# objective function
obj = .5 * cp.quad_form(beta, K) - np.ones(alpha.shape).T @ alpha

# constraints
const = [np.array(train_target.T) @ alpha == 0,
        -alpha <= np.zeros(alpha.shape),
        alpha <= 10*np.ones(shape=alpha.shape)]
prob = cp.Problem(cp.Minimize(obj), const)

In [12]:
result = prob.solve()

### recreate the hyperplane 

In [24]:
w = np.multiply(train_target, alpha.value).T @ train_df

In [14]:
S = (alpha.value > 1e-4).flatten()
b = train_target[S] - train_df[S] @ w
b = b[0]
# b = np.mean(b)

### define and test out the classifier
given the tuned parameters for the hyperplane

In [15]:
def classify(x):
    result = w @ x + b
    return np.sign(result)

In [16]:
correct = 0
incorrect = 0
validation_set = credit_df_norm.drop(target, axis=1)
predictions = []
for i, x in validation_set.iterrows():
    my_svm = classify(x)
    if my_svm==credit_df_norm[target].replace(0, -1)[i]: correct +=1
    else: incorrect +=1
    predictions.append(my_svm)
predictions = np.array(predictions)

In [31]:
print(f"fraction correct: {correct/(correct + incorrect)}")

fraction correct: 0.61


The above is the fraction of points for which my SVM correctly predicted the target 

### Compare to sklearn_svm

In [20]:
sklearn_svm = svm.SVC(C = 10, kernel='linear')
sklearn_svm.fit(train_df, train_target)

SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [21]:
sklearn_svm.score(credit_df_norm.drop(target, axis=1), credit_df_norm[target])

0.6075